# Analysing Trained detectors

## Importing libraries

In [10]:
import gc
import os
import sys
import math
import glob
import tqdm
import random
import numpy as np
from tqdm import tqdm
import time 
from time import sleep

In [11]:
import pandas as pd
import xml.etree.ElementTree as ET

In [12]:
import cv2
import dlib
from imutils import face_utils
from skimage.feature import hog
from skimage import data,exposure

In [13]:
import matplotlib 
%matplotlib inline
import matplotlib.pyplot as plt

In [14]:
from ipywidgets import widgets, interactive, fixed

## Paths

In [34]:
path_openface_xml = '/media/amogh/Stuff/OpenFace_Outputs/Openface_outputs/oface_combined_csv/openface_404_folders_75_confidence.xml'

In [16]:
path_detector_1_xml = '/home/amogh/cmu/detect_faces/custom_hog_output_1523.xml'

In [17]:
path_detector_2_xml = '/home/amogh/cmu/detect_faces/custom_hog_output_2523.xml'

In [18]:
path_detector_3_xml = '/home/amogh/cmu/detect_faces/custom_hog_output_3523.xml'

## Computation functions

In [19]:
def box_percent_covered (a,b):
    inner = a.intersect(b).area()
    if inner == 0:
        return 0
    return max(inner/a.area(), inner/b.area())

In [20]:
def drect_add(a,b):
    return dlib.drectangle(min(a.left(),b.left()),min(a.top(),b.top()),max(a.right(),b.right()),max(a.bottom(),b.bottom()))

In [21]:
def intersect(a,b,iou_thresh=0.5,percent_covered_thresh=1.0):
    
    inner = a.intersect(b).area()
    if (inner == 0):
        return false
    
    outer = drect_add(a,b).area()
    print("inner,outer,a.area,b.area: ", inner, outer, a.area(),b.area())
    if (inner/outer > iou_thresh or 
        inner/a.area() > percent_covered_thresh or 
        inner/b.area() > percent_covered_thresh):
        return True
    else:
        return False

In [22]:
a = dlib.drectangle(2,1,4.5,3)
b = dlib.drectangle(3,1,5,3)

In [23]:
type(max(1.,2.))

float

In [24]:
a.left()

2.0

In [25]:
intersect(a,b)

('inner,outer,a.area,b.area: ', 7.5, 12.0, 10.5, 9.0)


True

## Reading xml files

There are 2 xml files. One with the ground truth and the other with the detected. Given the name of the image, need to get the boxes list from one, and the box list from other xml.

In [35]:
def getListImages(xml_path):
    list_images = 
    return list_images

SyntaxError: invalid syntax (<ipython-input-35-92b68a0a2f28>, line 2)

In [49]:
def get_detected_boxes(image_file_name, path_detector_xml_file):
    """Get list of boxes detected"""
    
    tree = ET.parse(path_openface_xml)
    root = tree.getroot()
    for image in root.iter('image'):
        #image found
        if image.attrib['file'] == image_file_name:
            if (len(image) == 0):
                return []
            else:
                list_boxes = []
                for box_num, box in enumerate(image):
                    box_attribs = box.attrib
                    box_rect = dlib.drectangle((float)(box_attribs['left']),
                                               (float)(box_attribs['top']),
                                              ((float)(box_attribs['left'])+(float)(box_attribs['width'])),
                                              ((float)(box_attribs['top'])+(float)(box_attribs['height'])))
                    list_boxes.append(box_rect)
                return list_boxes
    print('image not found')
    return None

In [36]:
def debug():
    tree = ET.parse(path_openface_xml)
    root = tree.getroot()
    for 
    
debug()

In [ ]:
def getHitData(list_images, path_truth_xml_file, path_detector_xml_file):

    correct_hits = 0
    total_true_targets = 0
    missing_detections = 0
    total_hits = 0

    for image in list_images:

        detected_boxes = get_detected_boxes(image, path_detector_xml_file)
        true_boxes = get_detected_boxes(image, path_truth_xml_file)

        correct_hits += 
        total_true_targets = len(true_boxes)
        total_hits += len(detected_boxes)

    return total_hits, correct_hits, total_true_targets

    

def getAnalysis(total_hits, correct_hits, total_true_targets):
    
    if (total_hits == 0):
        precision = 1
    else:
        precision = correct_hits / total_hits
        
    if (total_true_targets == 0):
        recall = 1
    else:
        recall = correct_hits / total_true_targets